In [ ]:
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
from PIL import Image, ImageDraw
import ipywidgets as widgets
from IPython.display import display
from io import BytesIO

Import relevant libraries

In [ ]:
data = pd.read_csv('../data/train.csv')

data = np.array(data)
m, n = data.shape
np.random.shuffle(data) # shuffle before splitting into dev and training sets

Read dataset. Convert dataset in to numpy array. Then shuffle the data to ensure randomness for training

In [ ]:
data_dev = data[0:1000].T
Y_dev = data_dev[0]
X_dev = data_dev[1:n]
X_dev = X_dev / 255.

data_train = data[1000:m].T
Y_train = data_train[0]
X_train = data_train[1:n]
X_train = X_train / 255. # feature normalization
_,m_train = X_train.shape

Hier teilen wir das Dataset in ein Entwicklungssatz und ein Trainingssatz auf. Die Daten werden auch transponiert, um der erwarteten Eingabeform des neuronalen Netzwerks zu entsprechen. (Features als Spalten und Beispiele als Zeilen) Schließlich werden die Pixelwerte durch Teilen durch 255 normalisiert, sodass sie zwischen 0 und 1 liegen. Diese Normalisierung ist für die Stabilität des Trainings und eine schnellere Konvergenz unerlässlich.

In [ ]:
Y_train

In [ ]:
def init_params():
    W1 = np.random.rand(10, 784) - 0.5
    b1 = np.random.rand(10, 1) - 0.5
    W2 = np.random.rand(10, 10) - 0.5
    b2 = np.random.rand(10, 1) - 0.5
    return W1, b1, W2, b2

Die Funktion initialisiert die Gewichte (W1, W2) und Verzerrungen (b1, b2) unseres neuronalen Netzwerks. Die Initialisierung ist wichtig, da sie den Startpunkt unserer Optimierung beeinflusst.
W1 Form: 784 Eingabefeatures, 10 Neuronen in der ersten Schicht

In [ ]:
def ReLU(Z):
    return np.maximum(Z, 0)

def softmax(Z):
    A = np.exp(Z) / sum(np.exp(Z))
    return A

Hier sehen wir zwei Aktivierungsfunktionen:

    - ReLU (Rectified Linear Unit): Eine häufig verwendete Aktivierungsfunktion für versteckte Schichten. Sie ist rechenintensiv und hilft dabei, das Problem verschwindender Gradienten zu bewältigen.
    - Softmax: Wird in der Ausgabeschicht für Probleme der Mehrklassenklassifikation verwendet. Sie wandelt rohe Ausgabewerte in Wahrscheinlichkeiten für jede Klasse um

In [ ]:
def forward_prop(W1, b1, W2, b2, X):
    Z1 = W1.dot(X) + b1
    A1 = ReLU(Z1)
    Z2 = W2.dot(A1) + b2
    A2 = softmax(Z2)
    return Z1, A1, Z2, A2

Diese Funktion berechnet die Vorwärtsdurchlauf des Netzwerks. Sie berechnet anhand der aktuellen Gewichte und Verzerrungen Zwischenwerte (Z1, A1, Z2) und die endgültige Ausgabe A2.

Z1 = W1 (10, 784) * X (784, m) = (10, m) 

In [ ]:
def ReLU_deriv(Z):
    return Z > 0

def one_hot(Y):
    one_hot_Y = np.zeros((Y.size, Y.max() + 1))
    one_hot_Y[np.arange(Y.size), Y] = 1
    one_hot_Y = one_hot_Y.T
    return one_hot_Y

In [ ]:
def backward_prop(Z1, A1, Z2, A2, W1, W2, X, Y, m):
    one_hot_Y = one_hot(Y)
    dZ2 = A2 - one_hot_Y
    dW2 = 1 / m * dZ2.dot(A1.T)
    db2 = 1 / m * np.sum(dZ2)
    dZ1 = W2.T.dot(dZ2) * ReLU_deriv(Z1)
    dW1 = 1 / m * dZ1.dot(X.T)
    db1 = 1 / m * np.sum(dZ1)
    return dW1, db1, dW2, db2

Die Rückwärtspropagation berechnet, wie viel jedes Gewicht und jede Verzerrung zum Fehler beigetragen hat. Die zurückgegebenen Werte (dW1, db1, dW2, db2) stellen Gradienten dar, die die Richtung und Größe der zur Fehlerreduktion erforderlichen Änderungen anzeigen.

In [ ]:
def update_params(W1, b1, W2, b2, dW1, db1, dW2, db2, alpha):
    W1 = W1 - alpha * dW1
    b1 = b1 - alpha * db1    
    W2 = W2 - alpha * dW2  
    b2 = b2 - alpha * db2    
    return W1, b1, W2, b2

Diese Funktion aktualisiert die Gewichte und Verzerrungen des Netzwerks in Richtung des negativen Gradienten. Dieser iterative Prozess hilft dem Netzwerk, aus seinen Fehlern zu lernen. 'alpha' ist die Lernrate, die die Schrittgröße jeder Aktualisierung bestimmt.

In [ ]:
def get_predictions(A2):
    return np.argmax(A2, 0)


def get_accuracy(predictions, Y):
    print(predictions, Y)
    return np.sum(predictions == Y) / Y.size


def gradient_descent(X, Y, alpha, iterations):
    W1, b1, W2, b2 = init_params()
    for i in range(iterations):
        Z1, A1, Z2, A2 = forward_prop(W1, b1, W2, b2, X)
        dW1, db1, dW2, db2 = backward_prop(Z1, A1, Z2, A2, W1, W2, X, Y, m_train)
        W1, b1, W2, b2 = update_params(W1, b1, W2, b2, dW1, db1, dW2, db2, alpha)
        if i % 10 == 0:
            print("Iteration: ", i)
            predictions = get_predictions(A2)
            print(get_accuracy(predictions, Y))
    return W1, b1, W2, b2

Diese Funktion führt das Training unseres neuronalen Netzwerks durch. Es verwendet den Gradientenabstieg, um die Gewichte und Verzerrungen kontinuierlich zu aktualisieren. Am Ende des Trainings sollte unser Netzwerk besser auf die Daten abgestimmt sein und genauere Vorhersagen treffen können.

In [ ]:
W1, b1, W2, b2 = gradient_descent(X_train, Y_train, 0.10, 500)

In [ ]:
def make_predictions(X, W1, b1, W2, b2):
    _, _, _, A2 = forward_prop(W1, b1, W2, b2, X)
    predictions = get_predictions(A2)
    return predictions

def test_prediction(index, W1, b1, W2, b2):
    current_image = X_train[:, index, None]
    prediction = make_predictions(X_train[:, index, None], W1, b1, W2, b2)
    label = Y_train[index]
    print("Prediction: ", prediction)
    print("Label: ", label)
    
    current_image = current_image.reshape((28, 28)) * 255
    plt.gray()
    plt.imshow(current_image, interpolation='nearest')
    plt.show()

In [ ]:
test_prediction(0, W1, b1, W2, b2)
test_prediction(1, W1, b1, W2, b2)
test_prediction(2, W1, b1, W2, b2)
test_prediction(3, W1, b1, W2, b2)

In [ ]:
dev_predictions = make_predictions(X_dev, W1, b1, W2, b2)
get_accuracy(dev_predictions, Y_dev)

In [ ]:
canvas = widgets.Image(value=b'', format='png', width=280, height=280)

btn_clear = widgets.Button(description='Clear')
btn_check = widgets.Button(description='Check digit')

In [ ]:
img = Image.new('RGB', (280, 280), 'white')
draw = ImageDraw.Draw(img)

last_point = (0, 0)

In [ ]:
def handle_mouse_move(x, y):
    global last_point
    draw.line([last_point, (x, y)], fill='black', width=10)
    last_point = (x, y)
    update_canvas()

def handle_mouse_down(x, y):
    global last_point
    last_point = (x, y)

In [ ]:
def update_canvas():
    # Convert PIL image to bytes and set it as the canvas value
    canvas.value = img_to_byte_array(img)

def img_to_byte_array(img):
    """Convert PIL image to byte array."""
    img_byte_array = bytearray()
    with BytesIO() as output:
        img.save(output, format='PNG')
        img_byte_array = output.getvalue()
    return img_byte_array

def clear_canvas(button):
    global draw
    draw.rectangle([(0,0), img.size], fill='white')
    update_canvas()

In [ ]:
def check_digit(button):
    # Convert the image to the appropriate size (28x28) for the neural network
    small_img = img.resize((28, 28)).convert('L')  # Convert to grayscale
    input_array = np.array(small_img).reshape(784, 1) / 255.  # Normalize
    # Now you can use the input_array with your neural network for prediction.
    # e.g., prediction = your_nn_function(input_array)
    # For now, we'll just display the resized image to confirm it works
    display(small_img)

In [ ]:
canvas.on_msg(handle_mouse_move, 'mouse_move')
canvas.on_msg(handle_mouse_down, 'mouse_down')
btn_clear.on_click(clear_canvas)
btn_check.on_click(check_digit)

display(widgets.VBox([canvas, widgets.HBox([btn_clear, btn_check])]))